<a href="https://colab.research.google.com/github/57nety/GigaChat/blob/main/GigaChat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install gigachain
!pip install gigachain-community
!pip install unstructured[all-docs]
!pip install faiss-cpu
!pip install sentence-transformers
!pip install langchain_huggingface
!pip install "numpy==2.2.0"

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
  Using cached numpy-2.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gigachain 0.2.16 requires numpy<2,>=1; python_version < "3.12", but you have numpy 2.2.0 which is incompatible.
gigachain-communit

In [10]:
from langchain_community.document_loaders.word_document import UnstructuredWordDocumentLoader
from langchain_text_splitters.character import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
word_file = "little_prince.docx"
loader = UnstructuredWordDocumentLoader(word_file)
split_data = loader.load_and_split(text_splitter)

In [11]:
len(split_data)

93

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores.faiss import FAISS

In [13]:
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs,
                                  encode_kwargs=encode_kwargs)

vector_store = FAISS.from_documents(split_data, embedding=embedding)

In [14]:
embedding_retriever = vector_store.as_retriever(search_kwargs={"k":5})

In [15]:
from google.colab import userdata
auth = userdata.get('SBER_AUTH')

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import create_retrieval_chain

In [17]:
llm = GigaChat(credentials=auth, model='GigaChat-2-Max', verify_ssl_certs=False,
               profanity_check=False)
prompt = ChatPromptTemplate.from_template('''Ответь на вопрос пользователя.\
Используй только информацию из контекста. Если в контексте нет информации для \
ответа, сообщи об этом пользователю.
Контекст: {context}
Вопрос: {input}
Ответ:''')

In [18]:
document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

In [19]:
retrieval_chain = create_retrieval_chain(embedding_retriever, document_chain)

In [20]:
q1 = 'Как звали маленького принца?'
q2 = 'Кто автор произведения \"Маленький принц\"?'
q3 = 'Какого цвета луна?'
q4 = 'Сколько лет маленькому принцу?'

In [21]:
resp1 = retrieval_chain.invoke({'input':q1})
resp1

{'input': 'Как звали маленького принца?',
 'context': [Document(id='ac29ae4a-b4e2-4fd3-886d-5d167cb225dd', metadata={'source': 'little_prince.docx'}, page_content='—\xa0Будем друзьями, я совсем один,\xa0— сказал он.\n\n—\xa0Один… один… один… — откликнулось эхо.\n\n«Какая странная планета!\xa0— подумал Маленький принц.\xa0— Совсем сухая, вся в иглах и соленая. И у людей не хватает воображения. Они только повторяют то, что им скажешь… Дома у меня был цветок, моя краса и радость, и он всегда заговаривал первым».\n\nХX\n\nДолго шел Маленький принц через пески, скалы и снега и, наконец, набрел на дорогу. А все дороги ведут к людям.\n\n—\xa0Добрый день,\xa0— сказал он.\n\nПеред ним был сад, полный роз.\n\n—\xa0Добрый день,\xa0— отозвались розы.\n\nИ Маленький принц увидел, что все они похожи на его цветок.\n\n—\xa0Кто вы?\xa0— спросил он, пораженный.\n\n—\xa0Мы — розы,\xa0— отвечали розы.\n\n—\xa0Вот как… — промолвил Маленький принц.\n\nИ почувствовал себя очень-очень несчастным. Его красави

In [22]:
resp2 = retrieval_chain.invoke({'input':q2})
resp2

{'input': 'Кто автор произведения "Маленький принц"?',
 'context': [Document(id='ac29ae4a-b4e2-4fd3-886d-5d167cb225dd', metadata={'source': 'little_prince.docx'}, page_content='—\xa0Будем друзьями, я совсем один,\xa0— сказал он.\n\n—\xa0Один… один… один… — откликнулось эхо.\n\n«Какая странная планета!\xa0— подумал Маленький принц.\xa0— Совсем сухая, вся в иглах и соленая. И у людей не хватает воображения. Они только повторяют то, что им скажешь… Дома у меня был цветок, моя краса и радость, и он всегда заговаривал первым».\n\nХX\n\nДолго шел Маленький принц через пески, скалы и снега и, наконец, набрел на дорогу. А все дороги ведут к людям.\n\n—\xa0Добрый день,\xa0— сказал он.\n\nПеред ним был сад, полный роз.\n\n—\xa0Добрый день,\xa0— отозвались розы.\n\nИ Маленький принц увидел, что все они похожи на его цветок.\n\n—\xa0Кто вы?\xa0— спросил он, пораженный.\n\n—\xa0Мы — розы,\xa0— отвечали розы.\n\n—\xa0Вот как… — промолвил Маленький принц.\n\nИ почувствовал себя очень-очень несчастным

In [23]:
resp3 = retrieval_chain.invoke({'input':q3})
resp3

{'input': 'Какого цвета луна?',
 'context': [Document(id='8e7d8f53-c7ea-4b45-8ba6-b2bcc4b93b2f', metadata={'source': 'little_prince.docx'}, page_content='—\xa0Звезды очень красивые, потому что где-то там есть цветок, хоть его и не видно…\n\n—\xa0Да, конечно,\xa0— сказал я только, глядя на волнистый песок, освещенный луною.\n\n—\xa0И пустыня красивая… — прибавил Маленький принц.\n\nЭто правда. Мне всегда нравилось в пустыне. Сидишь на песчаной дюне. Ничего не видно. Ничего не слышно. И все же в тишине что-то светится…\n\n—\xa0Знаешь, отчего хороша пустыня?\xa0— сказал он.\xa0— Где-то в ней скрываются родники…\n\nЯ был поражен, вдруг я понял, что означает таинственный свет, исходящий от песков. Когда-то, маленьким мальчиком, я жил в старом-престаром доме — рассказывали, будто в нем запрятан клад. Разумеется, никто его так и не открыл, а может быть, никто никогда его и не искал. Но из-за него дом был словно заколдован: в сердце своем он скрывал тайну…\n\n—\xa0Да,\xa0— сказал я.\xa0— Будь 

In [24]:
resp4 = retrieval_chain.invoke({'input':q4})
resp4

{'input': 'Сколько лет маленькому принцу?',
 'context': [Document(id='b02ca656-2a4d-4068-8a79-202ccb362280', metadata={'source': 'little_prince.docx'}, page_content='—\xa0Ну да. Тридцать минут. Тридцать дней. Добрый вечер!\n\nИ он опять засветил фонарь.\n\nМаленький принц смотрел на фонарщика, и ему все больше нравился этот человек, который был так верен своему слову. Маленький принц вспомнил, как он когда-то переставлял стул с места на место, чтобы лишний раз поглядеть на закат солнца. И ему захотелось помочь другу.\n\n—\xa0Послушай,\xa0— сказал он фонарщику,\xa0— я знаю средство: ты можешь отдыхать, когда только захочешь…\n\n—\xa0Мне все время хочется отдыхать,\xa0— сказал фонарщик.\n\nВедь можно быть верным слову и все-таки ленивым.\n\n—\xa0Твоя планетка такая крохотная,\xa0— продолжал Маленький принц,\xa0— ты можешь обойти ее в три шага. И просто нужно идти с такой скоростью, чтобы все время оставаться на солнце. Когда захочется отдохнуть, ты просто все иди, иди… И день будет тянут